In [1]:
%run /home/bb/src/python/valkyrie/lib/valkyrie/apps/common/run_notebook_import.py

/media/veracrypt1/python/valkyrie/lib/valkyrie/apps/common/run_notebook_import.py:16: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


/media/veracrypt1/python/valkyrie/lib/valkyrie/ml/data.py:291: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [ ]:
torch.tensor

In [ ]:
sdate, edate = '2023-01-01', '2023-03-31'
instrs = ['BDM.BTC.USDT.FP']
freq = '1s'
ret_n_s = [60, 120, 300, 600]
root_dir = '/home/bb/data/BDM'
#########################################################
#DataMgr
#########################################################
data_mgr = DataMgr(sdate, edate, freq, root_dir, instrs = instrs, ret_n_s = ret_n_s)
df_res = data_mgr.get(instrs[0])
channels = ['bpx_last','apx_last'] + ['bq_last','aq_last','buy_qty_sum', 'sell_qty_sum'] 
n_channels = len(channels)
H, W = 16, 64
df2t2 = Df2T2(df_res, H, W, xcols = channels, ycol = 'mid_last_ret_300_n', yscaler = 1)

train_set, val_set = TrainValDataSet.create_train_valiation_pair(df2t2, 0.8)
dm = DataModule.from_dataset(train_set, val_set, 2048)

In [ ]:
from valkyrie.ml.net import Regressor, BatchNormAlongW


In [ ]:
class Residual(nn.Module):  #@save
    """The Residual block of ResNet models."""
    def __init__(self, num_channels, use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.LazyConv2d(num_channels, kernel_size=3, padding=1,
                                   stride=strides)
        self.conv2 = nn.LazyConv2d(num_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.LazyConv2d(num_channels, kernel_size=1,
                                       stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.LazyBatchNorm2d()
        self.bn2 = nn.LazyBatchNorm2d()

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.leaky_relu(Y, 0.1)
    
class ResNet(Regressor):
    def b1(self):
        return nn.Sequential(
            nn.LazyConv2d(64, kernel_size=3, stride=1, padding=1),
            nn.LazyBatchNorm2d(), nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1))    
        
    def block(self, num_residuals, num_channels, first_block=False):
        blk = []
        for i in range(num_residuals):
            if i == 0 and not first_block:
                blk.append(Residual(num_channels, use_1x1conv=True, strides=2))
            else:
                blk.append(Residual(num_channels))
        return nn.Sequential(*blk)
    
    def __init__(self, n_channel, height, arch, lr=0.1):
        super(ResNet, self).__init__('l1')
        self.save_hyperparameters()
        
        self.net = nn.Sequential(BatchNormAlongW(n_channel, height), self.b1())
        
        for i, b in enumerate(arch):
            self.net.add_module(f'b{i+2}', self.block(*b, first_block=(i==0)))
            
        self.net.add_module('last', nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(),
            nn.LazyLinear(1)))
        self.net.apply(d2l.init_cnn)
        
class ResNet18(ResNet):
    def __init__(self, n_channel, height, lr=0.1, num_classes=10):
        super().__init__(n_channel, height, ((2, 64), (2, 128), (2, 256), (2, 512)),
                       lr)

In [ ]:
res_net_18 = ResNet18(n_channels, H, lr = 1e-4)
X = df2t2[0][0]
ml_utils.layer_summary(res_net_18, X.shape)

In [ ]:
trainer = ml_modules.Trainer(max_epochs=100, log_fn = 'res_net_test.log')
with Timer('training'):
    trainer.fit(res_net_18, dm)